In [ ]:
from lxml import etree as ET
import dash
from dash import dcc, html
from dash import dash_table
import pandas as pd
import plotly.express as px

# Load and parse the XML
ccda_file = '../resources/CCD-Sample.xml'
tree = ET.parse(ccda_file)
root = tree.getroot()

# Function to extract patient demographics
def get_patient_demographics(root):
    namespace = {'hl7': 'urn:hl7-org:v3'}
    patient_info = root.find(".//hl7:patientRole", namespace)
    patient_data = {
        'Name': f"{patient_info.find('.//hl7:given', namespace).text} {patient_info.find('.//hl7:family', namespace).text}",
        'Gender': patient_info.find('.//hl7:administrativeGenderCode', namespace).get('displayName'),
        'Birth Date': patient_info.find('.//hl7:birthTime', namespace).get('value'),
        'Address': f"{patient_info.find('.//hl7:streetAddressLine', namespace).text}, {patient_info.find('.//hl7:city', namespace).text}, {patient_info.find('.//hl7:state', namespace).text}",
        'Phone': patient_info.find('.//hl7:telecom', namespace).get('value')
    }
    return patient_data

# Extract patient demographics
demographics = get_patient_demographics(root)

# Extract encounters
def get_encounters(root):
    namespace = {'hl7': 'urn:hl7-org:v3'}
    encounters = []
    for encounter in root.findall(".//hl7:encounter", namespace):
        data = {
            'Code': encounter.find('hl7:code', namespace).get('displayName'),
            'Effective Time': encounter.find('hl7:effectiveTime/hl7:low', namespace).get('value')
        }
        encounters.append(data)
    return pd.DataFrame(encounters)

encounters_df = get_encounters(root)

# Initialize Dash app
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("CCDA Document Explorer"),
    html.Div([
        html.H3("Patient Demographics"),
        html.Ul([html.Li(f"{key}: {value}") for key, value in demographics.items()])
    ]),
    html.Div([
        html.H3("Encounters"),
        dash_table.DataTable(
            id='encounters-table',
            columns=[{"name": i, "id": i} for i in encounters_df.columns],
            data=encounters_df.to_dict('records')
        )
    ])
])

if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)